In [1]:
from torch_geometric.datasets import Planetoid
import torch
import numpy as np
import networkx as nx

/home/jacklu/anaconda3/envs/pyg-gpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import utils
from data import get_dataset
# Load and pre-process data
adj, features, labels, idx_train, idx_val, idx_test = get_dataset('cora', 15)

# # change the format into networkx
G = nx.Graph()
edge_index = adj._indices().numpy()
G.add_edges_from(edge_index.T)
 
# # processed_features = utils.re_features(adj, features, args.w_len)  # return (N, hops+1, d)
processed_features = utils.get_token(G, features, 1000)  # return (N, hops+1, d)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


AttributeError: module 'utils' has no attribute 'get_token'

In [4]:
# Load the Cora dataset
dataset = Planetoid(root='data', name='Cora')
data = dataset[0]
# Convert the data object to a NetworkX graph
G = nx.Graph()
edge_index = data.edge_index.numpy()
G.add_edges_from(edge_index.T)
# Set the number of steps for the random walk
num_nodes = 5
# Loop over every node and perform a random walk   

concatenated_features_list = []
for i in range(len(G.nodes)):
    node_features = []
    for num_steps in range(1,2):
        neighbors =  get_k_layer_neighbors_feature(i, num_steps, G, num_nodes)
        num_neighbors = int(len(neighbors))
        # print the neighbros
        print(f"Node {i} has {num_neighbors - 1} neighbors in {num_steps} layer: {neighbors[1:num_neighbors+1]}")
        # add the neighbors
        neighbor_features = data.x[neighbors[1:]]
        temp_feature = data.x[neighbors[0]]
        node_features.append(torch.add(temp_feature, neighbor_features.mean(dim=0))) 
    # concate the feature
    concatenated_features = torch.cat(node_features, dim=0)
    concatenated_features = torch.cat([data.x[i], concatenated_features], dim=0)

    concatenated_features_list.append(concatenated_features)

Node 0 has 3 neighbors in 1 layer: [1862, 1701, 1865]
Node 1 has 3 neighbors in 1 layer: [654, 1, 2]
Node 2 has 3 neighbors in 1 layer: [1666, 1662, 2398]
Node 3 has 3 neighbors in 1 layer: [2544, 3, 2544]
Node 4 has 3 neighbors in 1 layer: [1016, 595, 1256]
Node 5 has 3 neighbors in 1 layer: [2546, 5, 1629]
Node 6 has 3 neighbors in 1 layer: [1416, 1008, 2575]
Node 7 has 3 neighbors in 1 layer: [208, 7, 208]
Node 8 has 3 neighbors in 1 layer: [281, 746, 2006]
Node 9 has 3 neighbors in 1 layer: [2614, 723, 2614]
Node 10 has 3 neighbors in 1 layer: [2545, 10, 2545]
Node 11 has 3 neighbors in 1 layer: [1839, 2453, 1839]
Node 12 has 3 neighbors in 1 layer: [1318, 12, 1001]
Node 13 has 3 neighbors in 1 layer: [1810, 568, 1116]
Node 14 has 3 neighbors in 1 layer: [2034, 2130, 2249]
Node 15 has 3 neighbors in 1 layer: [2367, 395, 2367]
Node 16 has 3 neighbors in 1 layer: [1632, 1756, 1632]
Node 17 has 3 neighbors in 1 layer: [1316, 17, 927]
Node 18 has 3 neighbors in 1 layer: [1786, 18, 1786

In [5]:
cora_d =  torch.stack(concatenated_features_list)
cora_d.shape[1]

2866

In [6]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [7]:
dataset[0].x.shape

torch.Size([2708, 1433])

In [13]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(cora_d.shape[1], 128)
        self.conv2 = GCNConv(128, dataset.num_classes)

    def forward(self, data):
        x, edge_index = cora_d.to(device), data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [15]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.7980
